# 导入需要的包

In [1]:
import xml.etree.ElementTree as ET
import re
import warnings
warnings.filterwarnings('ignore')

# 读入文件

In [2]:
tr1 = ET.ElementTree(file='./data/SS_back.vdx')

In [3]:
ans = {}
for node in tr1.iter():
    patten = '{.*?}'
    label = re.sub(patten,'',node.tag)
    if label == 'Pages':
        for ii in node.getchildren():
            new_label = re.sub(patten,'',ii.tag)
            if new_label == 'Page':
                tmp = {}
                tmp['ID'] = int(ii.attrib['ID']) #ID
                tmp['NameU'] = ii.attrib['NameU']
                tmp['ViewScale'] = ii.attrib['ViewScale'] #m名称
                tmp['ViewCenterX'] = ii.attrib['ViewCenterX']
                tmp['ViewCenterY'] = ii.attrib['ViewCenterY']
                ans[ii.attrib['ID']] = tmp

In [4]:
ans

{'4': {'ID': 4,
  'NameU': 'Page-1',
  'ViewScale': '0.34395833333333',
  'ViewCenterX': '7.896729254997',
  'ViewCenterY': '3.5508782556027'}}

# 定义函数

In [5]:
def Hex_to_RGB(hex):
    hex = hex.replace('#','')
    r = int(hex[0:2],16)
    g = int(hex[2:4],16)
    b = int(hex[4:6], 16)
    rgb = [r,g,b]
    return rgb

分析文件 SS_back.vdx,可以看出来：
   - 主要有以下几个标签：
       1. DocumentProperties
       2. Colors
       3. FaceNames
       4. StyleSheets
       5. DocumentSheet
       6. Masters
       7. Pages
       8. Windows
   - 但是看源c++文件，可以看出只针对：
       1. Colors
       2. FaceNames
       3. Masters
       4. StyleSheets
       5. Pages
   
   进行了解析遍历，暂时先按照源文档进行分析

In [ ]:
# vdx_pri.cpp
apply_XForm() masters 和 pages都有这个标签





# 定义vdxFile类

In [6]:
class vdxFile():
    '''
    vdxFile 主要存储解析出来的vdx各个标签的值
    
    '''
    def __init__(self,filename):
        self.tree = ET.ElementTree(file=filename)
        self.colors = {}
        self.facenames = {}
        self.masters = {}
        self.stylesheets = {}
        self.pages = {}
        for node in self.tree.iter():
            patten = '{.*?}'
            label = re.sub(patten,'',node.tag)
            if label == 'Colors':
                self.get_colors(node)
            if label == 'FaceNames':
                self.get_facenames(node)
            if label == 'Masters':
                self.get_masters(node)
            if label == 'StyleSheets':
                self.get_stylesheets(node)
            if label == 'Pages':
                self.get_pages(node)
    #获取颜色信息 Colors标签
    def get_colors(self,color_node):
        '''
        通过字典存储colors的信息，key为IX值，RGB通过转换，将16进制转换成10进制。
        例子:
        {'0': [0, 0, 0],
 '1': [255, 255, 255],
 '10': [0, 0, 128],
 '11': [128, 128, 0],...}
        
        '''
        for ii in color_node.getchildren():
            new_label = re.sub(patten,'',ii.tag)
            if new_label == 'ColorEntry':
                self.colors[ii.attrib['IX']] = Hex_to_RGB(ii.attrib['RGB'])
    # 获取字体类型FaceNames标签的值 
    def get_facenames(self, facenames_node):
        '''
        获取facenames的相关属性:
        例子:
        {
        '1': {'CharSets': '1614742015 -65536',
  'Flags': '357',
  'ID': 1,
  'Name': 'Arial Unicode MS',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-1 -369098753 63 0'},
 '10': {'CharSets': '536936959 539492352',
  'Flags': '327',
  'ID': 10,
  'Name': 'Estrangelo Edessa',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-452972801 -1073717157 41 0'},...
        }
        
        '''
        for ii in facenames_node.getchildren():
            new_label = re.sub(patten,'',ii.tag)
            if new_label == 'FaceName':
                tmp = {}
                tmp['ID'] = int(ii.attrib['ID']) #ID
                tmp['Name'] = ii.attrib['Name'] #m名称
                tmp['UnicodeRanges'] = ii.attrib['UnicodeRanges'] #unicode的范围
                tmp['CharSets'] = ii.attrib['CharSets'] #字符集合
                tmp['Panos'] = ii.attrib['Panos']
                tmp['Flags'] = ii.attrib['Flags']
                self.facenames[ii.attrib['ID']] = tmp
    def get_masters(self, master_node):
        '''
        获取master的相关属性
        例子:
        {'0': {'AlignName': '2',
  'BaseID': '{E6E7F788-3308-4917-8165-EE8196E285DA}',
  'Hidden': '0',
  'ID': 0,
  'IconSize': 1,
  'IconUpdate': '1',
  'MatchByName': '0',
  'Name': 'HMI_mtr_vlv_h_d',
  'NameU': 'HMI_mtr_vlv_h_d',
  'PatternFlags': 0,
  'Prompt': '{019D092E-0017-0000-8E40-00608CF305B2}',
  'UniqueID': '0'},...
  }
        '''
        for ii in master_node.getchildren():
            new_label = re.sub(patten,'',ii.tag)
            if new_label == 'Master':
                tmp = {}
                tmp['ID'] = int(ii.attrib['ID']) #ID
                tmp['AlignName'] = ii.attrib['AlignName']
                tmp['BaseID'] = ii.attrib['BaseID']
                tmp['Hidden'] = ii.attrib['Hidden']
                tmp['IconSize'] = int(ii.attrib['IconSize'])
                tmp['IconUpdate'] = ii.attrib['IconUpdate']
                tmp['MatchByName'] = ii.attrib['MatchByName']
                tmp['Name'] = ii.attrib['Name'] #m名称
                tmp['NameU'] = ii.attrib['NameU']
                tmp['PatternFlags'] = int(ii.attrib['PatternFlags'])
                tmp['Prompt'] = ii.attrib['UniqueID']
                tmp['UniqueID'] = ii.attrib['ID']
                self.masters[ii.attrib['ID']] = tmp
    def get_stylesheets(self, style_node):
        '''
        获取stylesheets的相关属性:
        例子:
        {'0': {'FillStyle_exists': '0',
  'ID': 0,
  'LineStyle_exists': '0',
  'Name': '无样式',
  'NameU': 'No Style',
  'TextStyle_exists': '0'},
 '1': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 1,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': '纯文本',
  'NameU': 'Text Only',
  'TextStyle': 3,
  'TextStyle_exists': '1'},...
  }
        '''
        for ii in style_node.getchildren():
            new_label = re.sub(patten,'',ii.tag)
            if new_label == 'StyleSheet':
                tmp = {}
                tmp['ID'] = int(ii.attrib['ID']) #ID
                try:
                    tmp['FillStyle'] = ii.attrib['FillStyle']
                    tmp['FillStyle_exists'] = '1'
                except:
                    tmp['FillStyle_exists'] = '0'
                try:
                    tmp['LineStyle'] = ii.attrib['LineStyle']
                    tmp['LineStyle_exists'] = '1'
                except:
                    tmp['LineStyle_exists'] = '0'
                tmp['Name'] = ii.attrib['Name'] #m名称
                tmp['NameU'] = ii.attrib['NameU']
                if 'TextStyle' in ii.attrib:
                    tmp['TextStyle'] = int(ii.attrib['TextStyle'])
                    tmp['TextStyle_exists'] ='1'
                else:
                    tmp['TextStyle_exists'] ='0'
                self.stylesheets[ii.attrib['ID']] = tmp
    def get_pages(self, pages_node):
        for ii in pages_node.getchildren():
            new_label = re.sub(patten,'',ii.tag)
            if new_label == 'Page':
                tmp = {}
                tmp['ID'] = int(ii.attrib['ID']) #ID
                tmp['NameU'] = ii.attrib['NameU']
                tmp['ViewScale'] = ii.attrib['ViewScale'] #m名称
                tmp['ViewCenterX'] = ii.attrib['ViewCenterX']
                tmp['ViewCenterY'] = ii.attrib['ViewCenterY']
                self.pages[ii.attrib['ID']] = tmp

# 测试结果

In [ ]:
mytest = vdxFile('./data/SS_back.vdx')

In [ ]:
mytest.colors
mytest.facenames
mytest.masters
mytest.stylesheets
mytest.pages

# vdx_out文件函数

In [ ]:
shapeCount = [-1]*24
vdx2ovsion = ["line","line","ellipse","polygon","dyntext","","","","","","","","","","","","template","polyline","","","","","",""]
vdx2ovsionClass = ["line","line","ellipse","Polygon","dyntext","","","","","","","","","","","","Template","Polyline","","","","","",""]
DISTANCE = 10
ID_TOOLS_LINE = 0
ID_TOOLS_POLYLINE = 1
ID_TOOLS_POLYLINE_JOIN = 31
ID_TOOLS_FULLPOLY_JOIN = 30
ID_TOOLS_FULLCIRCLE = 2
ID_TOOLS_FULLPOLY = 3
ID_TOOLS_TEXT = 4
ID_TOOLS_FULLRECT = 5
ID_TOOLS_BUTTON1 = 6
ID_TOOLS_BUTTON2 = 7
ID_TOOLS_SEGMENT7 = 8
ID_TOOLS_SEGMENT7 = 11
ID_TOOLS_FULLTANK = 9
ID_TOOLS_FULLPIE = 10
ID_TOOLS_FULLROUNDRECT = 11
ID_TOOLS_ARROW = 12
ID_TOOLS_FULLCURVE = 13
ID_TOOLS_FULLARC = 14
ID_TOOLS_TREND = 15


In [7]:
#写入文件头
def vdx_write_header(file_name):
    with open(file_name,'w+') as f:
        f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
        f.write("<hmiwidget>\n")
def write_line(file_name,dlFig,shapeID):
    if (dlFig.m_objtype >= 0):
        shapeCount[dlFig.m_objtype] += 1
    write_ovsion_object(file_name, dlFig , shapeCount[dlFig.m_objtype])


def write_ovsion_object(file_name, dlFig, shapeID):
    pass

def write_polyline (file_name, dlFig, shapeID):
    dlFigline = dlFig

    if (dlFig.m_pointX.size()>2):
        for i in range(dlFig.m_pointX.size()-2):
            dlFigline.m_pointX.clear()
            dlFigline.m_pointY.clear()
            dlFigline.m_pointX.push_back( dlFig.m_pointX.at(i))
            dlFigline.m_pointX.push_back(dlFig.m_pointX.at(i+1))
            dlFigline.m_pointY.push_back( dlFig.m_pointY.at(i))
            dlFigline.m_pointY.push_back(dlFig.m_pointY.at(i+1))
            if (dlFig.m_objtype >= 0):
                shapeCount[dlFig.m_objtype] += 1
            write_ovsion_object(file,dlFigline,shapeCount[dlFig.m_objtype])
    else:
        if (dlFig.m_objtype >= 0):
            shapeCount[dlFig.m_objtype] += 1
        write_ovsion_object(file,dlFig,shapeCount[dlFig.m_objtype])

def write_words(file_name,words):
    with open(file_name,'w+') as f:
        f.write(words)

In [8]:
def write_ovsion_object(file_name ,dlFig , shapeID ):
    if (shapeID == 0):
        if(dlFig.m_objtype==ID_TOOLS_FULLPOLY_JOIN):
            write_words(file_name,'<widget name="polygon" class="Polygon">\n')
        elif(dlFig.m_objtype==ID_TOOLS_POLYLINE_JOIN):
            write_words(file_name,'<widget name="{}" class="{}">\n'.format(vdx2ovsion[ID_TOOLS_POLYLINE],
                                                                     vdx2ovsionClass[ID_TOOLS_POLYLINE] ))
        else:
            write_words(file_name,'<widget name="{}" class="{}">\n'.format(vdx2ovsion[dlFig.m_objtype],
                                                                     vdx2ovsionClass[dlFig.m_objtype] ))
    else:
        if(dlFig.m_objtype==ID_TOOLS_FULLPOLY_JOIN):
            write_words(file_name,'<widget name="polygon" class="Polygon">\n')
        elif(dlFig.m_objtype==ID_TOOLS_POLYLINE_JOIN):
            write_words(file_name,'<widget name="{}" class="{}">\n'.format(vdx2ovsion[ID_TOOLS_POLYLINE],
                                                                     vdx2ovsionClass[ID_TOOLS_POLYLINE] ))
        else:
            write_words(file_name,'<widget name="{}" class="{}">\n'.format(vdx2ovsion[dlFig.m_objtype],
                                                                   vdx2ovsionClass[dlFig.m_objtype] ))
    write_words(file_name,'    <properties>\n') 
    left = 0
    top = 0 
    right = 0
    bottom = 0
    
    #获取坐标
    vdx2ovsionBackdrop( dlFig , left , top , right , bottom)
    if(dlFig.m_objtype==ID_TOOLS_FULLPOLY_JOIN):
        
        left = left - DISTANCE
        right = right + DISTANCE
        top = top - DISTANCE
        bottom = bottom + DISTANCE
    write_words(file_name,"        <property name=\"left\" type=\"short\" value=\"{}\"/>\n".format(left)) 
    write_words(file_name,"        <property name=\"top\" type=\"short\" value=\"{}\"/>\n".format(top)) 
    write_words(file_name,"        <property name=\"right\" type=\"short\" value=\"{}\"/>\n".format(right)) 
    write_words(file_name,"        <property name=\"bottom\" type=\"short\" value=\"{}\"/>\n".format(bottom)) 
    write_words(file_name, "        <property name=\"visible\" type=\"uchar\" value=\"1\"/>\n")
    write_words(file_name,"        <property name=\"enable\" type=\"uchar\" value=\"1\"/>\n")
    write_words(file_name,"        <property name=\"noredraw\" type=\"uchar\" value=\"0\"/>\n")
    write_words(file_name,"        <property name=\"flash\" type=\"uchar\" value=\"0\"/>\n")
    write_words(file_name,"        <property name=\"flashspeed\" type=\"ulong\" value=\"1\"/>\n")
    write_words(file_name,"        <property name=\"xoffset\" type=\"short\" value=\"0\"/>\n")
    write_words(file_name,"        <property name=\"yoffset\" type=\"short\" value=\"0\"/>\n")


    write_ovsion_private( file , dlFig);

    write_ovsion_trailer( file );

def vdx2ovsionBackdrop( dlFig,left,top, right,bottom):
    pointSize = dlFig.m_pointX.size()
    left = dlFig.m_pointX.at(0)
    right = dlFig.m_pointX.at(0)
    top = dlFig.m_pointY.at(0)
    bottom = dlFig.m_pointY.at(0)

    if (pointSize == 1):
        return
    for i in range(pointSize):
        left = min(dlFig.m_pointX.at(i),left) #x min
        right = max(dlFig.m_pointX.at(i),right) # x max
        bottom =   max(dlFig.m_pointY.at(i),bottom) # y max
        top = min(dlFig.m_pointY.at(i),top) # y min

    left = left * 1.2462
    right = right * 1.2462
    top = top * 1.3756
    bottom = bottom * 1.3756

def write_ovsion_private(file_name, dlFig):
    pass
def write_ovsion_trailer(file_name):
    write_words( file_name, "  </properties>\n")
    write_words( file_name, "  <dynamics/>\n")
    write_words( file_name, "  <events/>\n")
    write_words( file_name, " </widget>\n" )


In [9]:
def get_style(m_tpye):
    style = 1
    if dlFig.m_type == 0:
        style = 1
    elif dlFig.m_type == 1:
        style = 3
    elif dlFig.m_type == 2:
        style = 2
    elif dlFig.m_type == 3:
        style = 3;#modified by puxi
    elif dlFig.m_type == 4:
        style = 3
    elif dlFig.m_type == -1:
        style = 1
    else:
        style = 1
    return style

In [14]:
def write_ovsion_private( file_name, dlFig):
    if dlFig.m_objtype == ID_TOOLS_LINE:
        color = color2int(dlFig.m_fcolor)
        write_words( file_name, "        <property name=\"color\" type=\"ulong\" value=\"{}\"/>\n".format(color))
        write_words( file_name, "        <property name=\"linewidth\" type=\"ulong\" value=\"{}\"/>\n".format(dlFig.m_width) ,)
        direction =1 if (((dlFig.m_pointX.at(0)>dlFig.m_pointX.at(1)) and (dlFig.m_pointY.at(0)>dlFig.m_pointY.at(1)))
                         or ((dlFig.m_pointX.at(0)<dlFig.m_pointX.at(1)) and (dlFig.m_pointY.at(0)<dlFig.m_pointY.at(1)))) else 2
        write_words( file, "        <property name=\"direction\" type=\"ulong\" value=\"{}\"/>\n".format(direction) )
        style = get_style(dlFig.m_type)
        write_words ( file_name, "        <property name=\"style\" type=\"ulong\" value=\"{}\"/>\n".foramt(style))
    elif (dlFig.m_objtype == ID_TOOLS_POLYLINE) or (dlFig.m_objtype == ID_TOOLS_POLYLINE_JOIN):
        color = color2int(dlFig.m_fcolor)
        write_words( file_name, "        <property name=\"color\" type=\"ulong\" value=\"{}\"/>\n".format(color))
        write_words( file_name, "        <property name=\"linewidth\" type=\"ulong\" value=\"{}\"/>\n".format(dlFig.m_width))
        direction =1 if (((dlFig.m_pointX.at(0)> dlFig.m_pointX.at(1)) and (dlFig.m_pointY.at(0)>dlFig.m_pointY.at(1))) or
                         ((dlFig.m_pointX.at(0)<dlFig.m_pointX.at(1)) and (dlFig.m_pointY.at(0)<dlFig.m_pointY.at(1)))) else 2
        write_words ( file_name, "        <property name=\"direction\" type=\"ulong\" value=\"{}\"/>\n".format(direction) )
        style = style = get_style(dlFig.m_type)
        write_words( file_name, "        <property name=\"style\" type=\"ulong\" value=\"{}\"/>\n".format(style))
    elif (dlFig.m_objtype == ID_TOOLS_FULLPOLY) or (dlFig.m_objtype == ID_TOOLS_FULLPOLY_JOIN):
        left = 0
        top = 0
        right = 0
        bottom = 0
        vdx2ovsionBackdrop( dlFig,left,top,right,bottom)
        fcolor = color2int(dlFig.m_fcolor)
        bcolor = color2int(dlFig.m_bcolor)
        write_words ( file_name, "        <property name=\"linecolor\" type=\"ulong\" value=\"{}\"/>\n".format(fcolor) )
        write_words ( file_name, "        <property name=\"linewidth\" type=\"ulong\" value=\"{}\"/>\n".format(dlFig.m_width))
        #fprintf ( file, "        <property name=\"linelength\" type=\"ulong\" value=\"{}\"/>\n".format((right-left)))
        style = get_style(dlFig.m_type)
        write_words ( file_name, "        <property name=\"linestyle\" type=\"ulong\" value=\"{}\"/>\n".format(style))

        if dlFig.m_objtype ==ID_TOOLS_FULLPOLY:
            write_words ( file_name, "        <property name=\"joinStyle\" type=\"ulong\" value=\"2\"/>\n")
        elif dlFig.m_objtype == ID_TOOLS_FULLPOLY_JOIN:
            write_words ( file_name, "        <property name=\"joinStyle\" type=\"ulong\" value=\"1\"/>\n")
        else:
            write_words ( file_name, "        <property name=\"joinStyle\" type=\"ulong\" value=\"1\"/>\n")

        write_words ( file_name, "        <property name=\"fillcolor\" type=\"ulong\" value=\"%u\"/>\n" ,bcolor )
        if(dlFig.m_objtype==ID_TOOLS_FULLPOLY_JOIN):
            left=left-DISTANCE
            top=top-DISTANCE

        #多边形描点
        for i in range(dlFig.m_pointX.size()):
            str = QString()
            x = int(dlFig.m_pointX.at(i))
            y = int(dlFig.m_pointY.at(i))
            x = x * 1.2462 - left
            y = y * 1.3756 - top
            strPoints +=str
        write_words ( file_name, "        <property name=\"points\" type=\"string\" length=\"{}\" value=\"{}\"/>\n".format(strPoints.length() + 8,strPoints))
        write_words ( file_name, "        <property name=\"startcolor\" type=\"ulong\" value=\"{}\"/>\n".format(color2int(dlFig.m_startcolor)))
        write_words ( file_name, "        <property name=\"endcolor\" type=\"ulong\" value=\"{}\"/>\n" .format(color2int(dlFig.m_endcolor)))
        write_words ( file_name, "        <property name=\"direction\" type=\"ulong\" value=\"{}\"/>\n".format(dlFig.m_fillpattern))
    elif dlFig.m_objtype == ID_TOOLS_FULLCIRCLE:
        fcolor = color2int(dlFig.m_fcolor)
        bcolor = color2int(dlFig.m_bcolor)
        write_words ( file_name, "        <property name=\"edgecolor\" type=\"ulong\" value=\"{}\"/>\n".format(fcolor) )
        write_words ( file_name, "        <property name=\"fillcolor\" type=\"ulong\" value=\"{}\"/>\n".format(bcolor) )
        write_words ( file_name, "        <property name=\"fillstyle\" type=\"ulong\" value=\"{}\"/>\n".format(dlFig.m_brush) )
    elif dlFig.m_objtype ==ID_TOOLS_TEXT:
        left = 0
        top = 0
        right = 0
        bottom = 0
        vdx2ovsionBackdrop( dlFig,left,top,right,bottom)
        fcolor = color2int(dlFig.m_fcolor)
        bcolor = color2int(dlFig.m_bcolor)
        write_words ( file_name, "        <property name=\"color\" type=\"ulong\" value=\"{}\"/>\n".format(fcolor) )
#         QTextCodec *codec = QTextCodec::codecForName ( "utf8" )
#         const QString textO = dlFig.m_string
#         QString textT = codec->fromUnicode ( textO )
        textT.replace('\n',"\\n")
        #modified by puxi 文本去掉最后的回车
        if(textT.endsWith("\\n")):
            textT = textT.left(textT.length()-2)

#         const char *text = textT


        write_words ( file_name, "        <property name=\"text\" type=\"string\" length=\"{}\" value=\"{}\"/>\n".format
                     (dlFig.m_string.length()+1 , vdx_convert_xml_string(text)))
        if dlFig.m_type == 1:
            fontType="Times New Roman"
        elif dlFig.m_type == 3:
            fontType = "Courier New"
        elif dlFig.m_type == 4:
            fontType = "Lucida Console"
        elif dlFig.m_type == 5:
            fontType = "宋体"
        elif dlFig.m_type == 7:
            fontType = "仿宋"
        elif dlFig.m_type == 9:
            fontType = "Arial"
        else:
            fontType="Arial"
        #modified by puxi ,for the existed debug the fontType_ value is 0 but not null
        if(fontType_ != "" and not strcmp(fontType_ , "0") and fontType_.length() > 3):
            
            fontType = fontType_

        write_words ( file_name, "        <property name=\"fontname\" type=\"string\" length=\"{}\" value=\"{}\"/>\n".format(fontType.length()+1,fontType.latin1()) )#m_type
        write_words ( file_name, "        <property name=\"pointsize\" type=\"long\" value=\"{}\"/>\n".format(dlFig.m_fontname) ) #m_fontname
        fontbold = 0
        fontitalic = 0
        fontunderline = 0
        align = 0
        angle=0
        valign=0
        angle=dlFig.angle
        valign = dlFig.m_valign
        fontStyle2Ovation(dlFig.m_width,fontbold,fontitalic,fontunderline)
        fontalign2Ovation(dlFig.m_brush,align)

        write_words ( file_name, "        <property name=\"fontbold\" type=\"uchar\" value=\"%d\"/>\n" .format(fontbold))
        write_words ( file_name, "        <property name=\"fontitalic\" type=\"uchar\" value=\"%d\"/>\n" .format(fontitalic))
        write_words ( file_name, "        <property name=\"fontunderline\" type=\"uchar\" value=\"%d\"/>\n" .format(fontunderline) )
        write_words ( file_name, "        <property name=\"fontstrikeout\" type=\"uchar\" value=\"0\"/>\n"  )
        write_words ( file_name, "        <property name=\"align\" type=\"ulong\" value=\"%d\"/>\n".format(align))#m_brush
        write_words ( file_name, "        <property name=\"VerticalAlign\"  type=\"ulong\" value=\"%d\"/>\n".format(valign))#m_brush
        write_words ( file_name, "        <property name=\"angle\"  type=\"ulong\" value=\"1\"/>\n" )#m_brush

#     case ID_TOOLS_FULLRECT://1 polygon
#         break
#     case ID_TOOLS_BUTTON1://3 polygons
#         break
#     case ID_TOOLS_BUTTON2://3 polygons
#         break
#     case ID_TOOLS_SEGMENT7://7 polygons
#         break
#     case ID_TOOLS_FULLTANK://1 polygon
#         break
#     case ID_TOOLS_FULLPIE://1 polygons
#         break
#     case ID_TOOLS_FULLROUNDRECT:
#         break
#     case ID_TOOLS_ARROW:
#         break
#     case ID_TOOLS_FULLCURVE:
#         break
#     case ID_TOOLS_FULLARC:
#         break
#     case ID_TOOLS_TREND:
#         break
    elif dlFig.m_objtype == 17:
        left = 0
        top = 0
        right = 0
        bottom = 0
        vdx2ovsionBackdrop( dlFig,left,top,right,bottom)
        fcolor = color2int(dlFig.m_fcolor)
        bcolor = color2int(dlFig.m_bcolor)
        write_words ( file_name, "        <property name=\"linecolor\" type=\"ulong\" value=\"{}\"/>\n".format(fcolor) )
        write_words ( file_name, "        <property name=\"linewidth\" type=\"ulong\" value=\"{}\"/>\n".format(dlFig.m_width))
        style = get_style(dlFig.m_type)
        write_words ( file_name, "        <property name=\"linestyle\" type=\"ulong\" value=\"{}\"/>\n".format(style))
        strPoints = ''
        for i in range(dlFig.m_pointX.size()):
            str = "(%d,%d)".format(dlFig.m_pointX.at(i)-left,(dlFig.m_pointY.at(i)-top))
            strPoints+=str
        write_words ( file_name, "        <property name=\"points\" type=\"string\" length=\"%d\" value=\"%s\"/>\n".format(strPoints.length()+8,strPoints))

    else:
        print( "unrecognized shape ignored!" )

In [15]:
# 定义DL_FigureData 类
class DL_FigureData():
    def __init__(self):
        pass
class QString():
    def __init__(self):
        pass